In [18]:
import shutil
import os
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession

# ---------- CONFIGURAÇÃO DO SPARK COM DELTA ----------
builder = (
    SparkSession.builder.appName("DeltaReset")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

# ---------- LISTA DE TABELAS E CAMINHOS ----------
tabelas = {
    "carro_delta": "/home/hian/delta-ice/delta-tables/carro_delta",
    "proprietario_delta": "/home/hian/delta-ice/delta-tables/proprietario_delta"
}

# ---------- DROPAR TABELAS DO CATÁLOGO ----------
for nome in tabelas.keys():
    spark.sql(f"DROP TABLE IF EXISTS {nome}")
    print(f"🗑 Tabela '{nome}' removida do catálogo (se existia).")

# ---------- APAGAR PASTAS FÍSICAS ----------
for nome, path in tabelas.items():
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"✅ Pasta da tabela '{nome}' apagada.")
    else:
        print(f"⚠️ Pasta da tabela '{nome}' não existe. Pulando...")

# ---------- RECRIAR AS TABELAS ----------
spark.sql(f"""
CREATE TABLE carro_delta (
    sk_carro INT,
    placa STRING,
    modelo STRING,
    ano INT,
    valor DOUBLE
) USING delta
LOCATION '{tabelas['carro_delta']}'
""")

spark.sql(f"""
CREATE TABLE proprietario_delta (
    sk_proprietario INT,
    nome STRING,
    cpf STRING
) USING delta
LOCATION '{tabelas['proprietario_delta']}'
""")

# ---------- INSERIR DADOS INICIAIS ----------
spark.sql("""
INSERT INTO carro_delta VALUES
(1, 'ABC1234', 'Gol', 2015, 25000),
(2, 'XYZ9876', 'Onix', 2019, 45000)
""")

spark.sql("""
INSERT INTO proprietario_delta VALUES
(1, 'João Silva', '123.456.789-00'),
(2, 'Maria Oliveira', '987.654.321-00')
""")

print("✅ Ambiente Delta totalmente resetado e tabelas recriadas com dados de exemplo!")


25/10/02 15:50:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


🗑 Tabela 'carro_delta' removida do catálogo (se existia).
🗑 Tabela 'proprietario_delta' removida do catálogo (se existia).
⚠️ Pasta da tabela 'carro_delta' não existe. Pulando...
⚠️ Pasta da tabela 'proprietario_delta' não existe. Pulando...


25/10/02 15:50:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

✅ Ambiente Delta totalmente resetado e tabelas recriadas com dados de exemplo!


In [19]:
# Atualiza o valor do carro com modelo 'Gol'
spark.sql("""
UPDATE carro_delta
SET valor = 26000
WHERE modelo = 'Gol'
""")

# Verificar resultado
spark.sql("SELECT * FROM carro_delta").show()


25/10/02 15:51:32 WARN UpdateCommand: Could not validate number of records due to missing statistics.
                                                                                

+--------+-------+------+----+-------+
|sk_carro|  placa|modelo| ano|  valor|
+--------+-------+------+----+-------+
|       2|XYZ9876|  Onix|2019|45000.0|
|       1|ABC1234|   Gol|2015|26000.0|
+--------+-------+------+----+-------+



In [20]:
# Deleta o carro com modelo 'Onix'
spark.sql("""
DELETE FROM carro_delta
WHERE modelo = 'Onix'
""")

# Verificar resultado
spark.sql("SELECT * FROM carro_delta").show()


25/10/02 15:52:41 WARN DeleteCommand: Could not validate number of records due to missing statistics.
                                                                                

+--------+-------+------+----+-------+
|sk_carro|  placa|modelo| ano|  valor|
+--------+-------+------+----+-------+
|       1|ABC1234|   Gol|2015|26000.0|
+--------+-------+------+----+-------+

